In [1]:
%pip install numpy scikit-learn

Note: you may need to restart the kernel to use updated packages.


### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [2]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [3]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
tfidfvect = TfidfVectorizer()

In [4]:
# en el atributo `data` accedemos al texto
print(newsgroups_train.data[0])

I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.


In [5]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [6]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'Cantidad de documentos: {X_train.shape[0]}')
print(f'Tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
Cantidad de documentos: 11314
Tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [7]:
# una vez fiteado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [8]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [9]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [10]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [11]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [78]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]
print(cossim)    

[0.1382319  0.1067036  0.23029327 ... 0.12320753 0.08765353 0.04415046]


In [72]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.70930477, 0.67474953, ..., 0.        , 0.        ,
       0.        ])

In [73]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([ 4811,  6635,  4253, ...,  1534, 10055,  4750], dtype=int64)

In [79]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]
print(mostsim)

[6635 4253 3596 4271 3746]


In [75]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.politics.misc'

In [76]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


### Modelo de clasificación Naïve Bayes

In [18]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [19]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [20]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

### Consigna del desafío 1

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.


## 1.

In [94]:
import random

random_document_indexes = random.sample(range(X_train.shape[0]), 5)
cossim_list = []

for j, index in enumerate(random_document_indexes):

    cossim = cosine_similarity(X_train[index], X_train)[0]
    mostsim = np.argsort(cossim)[::-1][1:6]
    cossim_list.append(mostsim)
    
    print("Índice del documento original:", index)
    print("Documento original:", newsgroups_train.target_names[y_train[index]])
    for i in mostsim:
        print("Similar a:", newsgroups_train.target_names[y_train[i]])

Índice del documento original: 6630
Documento original: comp.windows.x
Similar a: comp.windows.x
Similar a: comp.windows.x
Similar a: comp.windows.x
Similar a: comp.windows.x
Similar a: comp.windows.x
Índice del documento original: 8336
Documento original: talk.politics.misc
Similar a: talk.politics.misc
Similar a: talk.politics.misc
Similar a: talk.politics.misc
Similar a: talk.politics.misc
Similar a: sci.crypt
Índice del documento original: 2169
Documento original: rec.sport.hockey
Similar a: rec.sport.hockey
Similar a: rec.sport.hockey
Similar a: rec.sport.hockey
Similar a: rec.sport.hockey
Similar a: rec.sport.hockey
Índice del documento original: 1272
Documento original: comp.os.ms-windows.misc
Similar a: comp.graphics
Similar a: sci.med
Similar a: comp.sys.ibm.pc.hardware
Similar a: comp.sys.ibm.pc.hardware
Similar a: comp.windows.x
Índice del documento original: 8067
Documento original: rec.sport.baseball
Similar a: rec.sport.baseball
Similar a: rec.sport.baseball
Similar a: re

A continuación se analizan los 5 documentos más similares al cuarto de ellos (el documento nro 1272)

In [120]:
print(random_document_indexes[3])
print(cossim_list[3])
print(newsgroups_train.target_names[y_train[random_document_indexes[3]]])
print(newsgroups_train.data[random_document_indexes[3]])

1272
[2429 8719 2483 5917 2286]
comp.os.ms-windows.misc
Hello,

I am looking for a Xterm emulator which runs under windows 3.1.

Please reply via E-mail.

Thanks.

Chera Bekker


In [130]:
print(newsgroups_train.target_names[y_train[cossim_list[3][0]]])
print(newsgroups_train.data[cossim_list[3][0]])

comp.graphics
Hello,


In [131]:
print(newsgroups_train.target_names[y_train[cossim_list[3][1]]])
print(newsgroups_train.data[cossim_list[3][1]])

sci.med
Hello,


In [123]:
print(newsgroups_train.target_names[y_train[cossim_list[3][2]]])
print(newsgroups_train.data[cossim_list[3][2]])

comp.sys.ibm.pc.hardware
I am looking for information about this drive.  Switch settings, geometry..etc.

Conner CP3204F

Please reply via e-mail.  Many thanks in advance!



In [127]:
print(newsgroups_train.target_names[y_train[cossim_list[3][3]]])
print(newsgroups_train.data[cossim_list[3][3]])

comp.sys.ibm.pc.hardware
What do I need to do to configure this drive as a slave?
Model# CP30101G

Please reply via e-mail. Thanks!!


In [128]:
print(newsgroups_train.target_names[y_train[cossim_list[3][4]]])
print(newsgroups_train.data[cossim_list[3][4]])

comp.windows.x
The situation: running a Fortran executable that creats an Xterm.  An option
in the menu contained in the Xterm runs a Fortran subroutine that creats
a Tektronics-mode Xterm for displaying some graphics.  Typing a <RET> while
the Tektronics-mode Xterm (future reference to this will be "Tek-term") is
active makes the Xterm read future keystrokes but does not shift input focus
to the Xterm (I'm basing that statement on the fact that the border of the 
Tek-term stays the highlighted color and that the Tek-term stays on top of
Xterm.  If what I'm describing isn't input focus, let me know.)  

The xterm can be brought to the top by clicking the mouse button on it.

Subsequent selections from the Xterm of the menu item that displays graphics 
highlihts the border and displays the updated graphics, but does not move 
the Tek-term to the top.  If the Tek-term has been iconized to conserve 
screen space, it stays an icon.


The desired behavior: one of two options - 

    1. Whic

Los primeros dos parecería ser prácticamente azar, debido a que el contenido de dichos documentos es muy pequeño. El tercer y cuarto documento, a pesar de no ser de la misma categoría, contienen un contenido que está estrechamente relacionado. Por último, el quinto resultado es correcto.

## 2.

In [22]:
#Complement Naive Bayes
from sklearn.model_selection import GridSearchCV

complement_nb = ComplementNB()
alpha_values = [0.01, 0.05, 0.1, 0.5, 1.0, 5.0, 10.0]

param_grid = {'alpha': alpha_values}
grid_search = GridSearchCV(complement_nb, param_grid, scoring='f1_macro', cv=5, n_jobs=-1)

grid_search.fit(X_train, y_train)

best_alpha = grid_search.best_params_['alpha']
print(f"Mejor alpha encontrado: {best_alpha}")

best_complement_nb = ComplementNB(alpha=best_alpha)
best_complement_nb.fit(X_train, y_train)

y_pred_best_cnb = best_complement_nb.predict(X_test)
best_f1 = f1_score(y_test, y_pred_best_cnb, average='macro')
print(f"Mejor F1-score en test: {best_f1}")

Mejor alpha encontrado: 0.1
Mejor F1-score en test: 0.6953652590540836


In [23]:
#Multinomial Naive Bayes

multinomial_nb = MultinomialNB()
alpha_values = [0.01, 0.05, 0.1, 0.5, 1.0, 5.0, 10.0]

param_grid = {'alpha': alpha_values}
grid_search = GridSearchCV(multinomial_nb, param_grid, scoring='f1_macro', cv=5, n_jobs=-1)

grid_search.fit(X_train, y_train)

best_alpha = grid_search.best_params_['alpha']
print(f"Mejor alpha encontrado: {best_alpha}")

best_multinomial_nb = ComplementNB(alpha=best_alpha)
best_multinomial_nb.fit(X_train, y_train)

y_pred_best_mnb = best_multinomial_nb.predict(X_test)
best_f1 = f1_score(y_test, y_pred_best_mnb, average='macro')
print(f"Mejor F1-score en test: {best_f1}")

Mejor alpha encontrado: 0.01
Mejor F1-score en test: 0.668895083530568


In [24]:
tfidfvect_2 = TfidfVectorizer(stop_words='english', max_df=0.8, min_df=3, ngram_range=(1,2), sublinear_tf=True, norm='l2')

X_train_2 = tfidfvect_2.fit_transform(newsgroups_train.data)
X_test_2 = tfidfvect_2.transform(newsgroups_test.data)

complement_nb_2 = ComplementNB(alpha=0.1)
complement_nb_2.fit(X_train_2, y_train)

y_pred_cnb_2 = complement_nb_2.predict(X_test_2)
score = f1_score(y_test, y_pred_cnb_2, average='macro')
print(f"Mejor F1-score en test: {score}")

Mejor F1-score en test: 0.6986671237912787


In [25]:
multinomial_nb_2 = MultinomialNB(alpha=0.01)
multinomial_nb_2.fit(X_train_2, y_train)

y_pred_mnb_2 = multinomial_nb_2.predict(X_test_2)
score = f1_score(y_test, y_pred_mnb_2, average='macro')
print(f"Mejor F1-score en test: {score}")

Mejor F1-score en test: 0.678080111631851


Se optó por utilizar los alpha encontrados anteriormente, en lugar de volver a buscarlos. Si la precisión fuera el objetivo final, debería de volver a buscarse el alpha con el vocabulario generado nuevamente.

## 3.

In [138]:
X_train_tfidf_T = X_train_2.T  # o X_train_2.transpose()
print(type(X_train_tfidf_T))
print(f'shape: {X_train_tfidf_T.shape}')
print(f'Cantidad de palabras: {X_train_tfidf_T.shape[0]}')

<class 'scipy.sparse._csc.csc_matrix'>
shape: (62739, 11314)
Cantidad de palabras: 62739


In [146]:
words = tfidfvect_2.get_feature_names_out()

selected_words = ["computer", "network", "space", "car", "science"]
word_indices = [np.where(words == word)[0][0] for word in selected_words if word in words]
cosine_similarities = cosine_similarity(X_train_tfidf_T)

for idx, word in zip(word_indices, selected_words):
    if idx is not None:
        similar_indices = np.argsort(cosine_similarities[idx])[::-1][1:6]
        similar_words = [words[i] for i in similar_indices]
        print(f'Palabras más similares a "{word}": {similar_words}')

Palabras más similares a "computer": ['computer science', 'computer graphics', 'turn computer', 'new computer', 'computer equipment']
Palabras más similares a "network": ['products using', 'utilizing', 'network don', 'pc network', 'pc home']
Palabras más similares a "space": ['space station', 'sci space', 'nasa', 'space shuttle', 'shuttle']
Palabras más similares a "car": ['new car', 'bought car', 'cars', 'car like', 'car car']
Palabras más similares a "science": ['computer science', 'scientific', 'science collection', 'competing religious', 'psychology like']


Aparecen construcciones de más de una palabra por haber utilizado la opción de ngram_range=(1,2).

Las palabras más similares reflejan asociaciones semánticas esperadas.

Por ejemplo, "computer" está estrechamente relacionado con "computer science", "computer graphics", y "new computer", lo que sugiere que el modelo ha capturado términos comunes dentro del dominio de la informática.
Lo mismo ocurre con "space", que se asocia con "space station", "nasa", y "space shuttle"..
Algunas similitudes parecen basadas en coocurrencias en frases comunes

En el caso de "car", términos como "new car", "bought car", y "car like" reflejan expresiones cotidianas, más que sinónimos o términos técnicos relacionados con automóviles. Algo similar ocurre con "network", donde aparecen frases como "network don" y "pc network", lo que sugiere que los términos están agrupados por contexto en lugar de significado exacto. Algunas asociaciones pueden no ser completamente interpretables, or ejemplo "car car".

En "science", encontramos términos esperados como "computer science" y "scientific", pero también "competing religious", lo cual podría indicar que en los textos hay discusiones sobre la relación entre ciencia y religión.
En "network", términos como "products using" o "utilizing" parecen menos intuitivos y podrían ser ruido en la representación del modelo.

Especialmente en áreas técnicas ("computer", "science", "space") el modelo encuentra términos bien relacionados.
Sin embargo, en palabras más generales como "car", el modelo parece estar agrupando términos por proximidad textual en lugar de significado puro.